In [9]:
import os
import numpy as np
import pandas as pd
from module import in_fun
from matplotlib import pyplot as plt
from module.torch import BinaryClassifier

beacon_list = [1, 2, 3, 4, 10, 6, 7, 8, 9]

%matplotlib widget

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [11]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

print(device)

cpu


In [12]:
net_abs = BinaryClassifier(input_size = 9)
net_diff = BinaryClassifier(input_size = 9)
net_each = BinaryClassifier(input_size = 18)
net_abs.load_state_dict(torch.load("./model/abs"))
net_diff.load_state_dict(torch.load("./model/diff"))
net_each.load_state_dict(torch.load("./model/each"))

<All keys matched successfully>

In [13]:
data_path = "data/210317/"
file_list = os.listdir(data_path)
file_list_csv = [file for file in file_list if file.endswith(".csv")]
print(file_list_csv)

['1239.csv', 'toy01.csv', 'toy02.csv']


In [14]:
data_list = []
for file_csv in file_list_csv:
    data_list.append(in_fun.parser(
    pd.read_csv(
        data_path + file_csv,
        header=None
    ).loc[5000:].to_numpy(), beacon_list))
    print(np.shape(data_list[-1]))    

(71434, 32)
(72365, 32)
(68615, 32)


In [15]:
gathered_list = []
gather_num = 250
for data in data_list:
    cur_gathered = np.zeros([np.shape(data)[0] // gather_num, 9])
    for row in range(np.shape(cur_gathered)[0]):
        cur_gathered[row] = np.max(data[row * gather_num : row * gather_num + gather_num, -9:], axis=0)
    cur_gathered += 200
    cur_gathered = cur_gathered / 200
    gathered_list.append(cur_gathered)
    print(np.shape(gathered_list[-1]))

(285, 9)
(289, 9)
(274, 9)


In [16]:
matched_same = []
for gathered in gathered_list:
    matched_same.append(in_fun.matcher(gathered, gathered))
matched_same.append(in_fun.matcher(gathered_list[1], gathered_list[2]))
matched_same.append(in_fun.matcher(gathered_list[2], gathered_list[1]))
    
same_abs = np.concatenate([same[0] for same in matched_same])
same_diff = np.concatenate([same[1] for same in matched_same])
same_each = np.concatenate([same[2] for same in matched_same])
print(np.shape(same_abs))

(398194, 9)


In [17]:
matched_diff = []
matched_diff.append(in_fun.matcher(gathered_list[0], gathered_list[1]))
matched_diff.append(in_fun.matcher(gathered_list[1], gathered_list[0]))
matched_diff.append(in_fun.matcher(gathered_list[0], gathered_list[2]))
matched_diff.append(in_fun.matcher(gathered_list[2], gathered_list[0]))

diff_abs = np.concatenate([diff[0] for diff in matched_diff])
diff_diff = np.concatenate([diff[1] for diff in matched_diff])
diff_each = np.concatenate([diff[2] for diff in matched_diff])
print(np.shape(diff_each))

(320910, 18)


In [18]:
same_tensor_abs = torch.from_numpy(same_abs).float()
same_tensor_diff = torch.from_numpy(same_diff).float()
same_tensor_each = torch.from_numpy(same_each).float()
diff_tensor_abs = torch.from_numpy(diff_abs).float()
diff_tensor_diff = torch.from_numpy(diff_diff).float()
diff_tensor_each = torch.from_numpy(diff_each).float()

same_result_abs = net_abs(same_tensor_abs)
same_result_diff = net_diff(same_tensor_diff)
same_result_each = net_each(same_tensor_each)

diff_result_abs = net_abs(diff_tensor_abs)
diff_result_diff = net_diff(diff_tensor_diff)
diff_result_each = net_each(diff_tensor_each)

In [19]:
print(len(same_result_abs))
print(f"same correct : {len(same_result_abs[same_result_abs>=0.5])}")
print(f"same wrong : {len(same_result_abs[same_result_abs<=0.5])}")
print(len(diff_result_abs))
print(f"diff correct : {len(diff_result_abs[diff_result_abs<=0.5])}")
print(f"diff wrong : {len(diff_result_abs[diff_result_abs>=0.5])}")

398194
same correct : 363229
same wrong : 34965
320910
diff correct : 320910
diff wrong : 0


In [20]:
print(len(same_result_diff))
print(f"same correct : {len(same_result_diff[same_result_diff>=0.5])}")
print(f"same wrong : {len(same_result_diff[same_result_diff<=0.5])}")
print(len(diff_result_diff))
print(f"diff correct : {len(diff_result_diff[diff_result_diff<=0.5])}")
print(f"diff wrong : {len(diff_result_diff[diff_result_diff>=0.5])}")

398194
same correct : 381060
same wrong : 17134
320910
diff correct : 320347
diff wrong : 563


In [21]:
print(len(same_result_each))
print(f"same correct : {len(same_result_each[same_result_each>=0.5])}")
print(f"same wrong : {len(same_result_each[same_result_each<=0.5])}")
print(len(diff_result_each))
print(f"diff correct : {len(diff_result_each[diff_result_each<=0.5])}")
print(f"diff wrong : {len(diff_result_each[diff_result_each>=0.5])}")

398194
same correct : 380569
same wrong : 17625
320910
diff correct : 320347
diff wrong : 563


In [22]:
print(np.shape(same_result_each.reshape(-1)))

torch.Size([398194])


In [23]:
print(same_each[same_result_abs.reshape(-1)<=0.5][:10,:9])
print(same_each[same_result_abs.reshape(-1)<=0.5][:10,9:])

[[0.    0.    0.    0.    0.    0.    0.    0.575 0.605]
 [0.    0.    0.    0.    0.    0.    0.    0.575 0.605]
 [0.    0.    0.    0.    0.    0.    0.    0.575 0.605]
 [0.    0.    0.    0.    0.    0.    0.    0.575 0.605]
 [0.    0.    0.    0.    0.    0.    0.    0.575 0.605]
 [0.    0.    0.    0.    0.    0.    0.    0.575 0.605]
 [0.    0.    0.    0.    0.    0.    0.    0.575 0.605]
 [0.    0.    0.    0.    0.    0.    0.    0.575 0.605]
 [0.    0.    0.    0.    0.    0.    0.    0.575 0.605]
 [0.    0.    0.    0.    0.    0.    0.    0.575 0.605]]
[[0.    0.    0.    0.    0.    0.    0.    0.    0.605]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.585]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.605]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.61 ]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.58 ]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.61 ]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.6  ]
 [0.    0.    0.    0.    0.  

In [24]:
print(same_each[same_result_each.reshape(-1)<=0.5][:10,:9])
print(same_each[same_result_each.reshape(-1)<=0.5][:10,9:])

[[0.    0.    0.    0.    0.    0.    0.    0.575 0.605]
 [0.    0.    0.    0.    0.    0.    0.    0.575 0.605]
 [0.    0.    0.    0.    0.    0.    0.    0.575 0.605]
 [0.    0.    0.    0.    0.    0.    0.    0.575 0.605]
 [0.    0.    0.    0.    0.    0.    0.    0.575 0.605]
 [0.    0.    0.    0.    0.    0.    0.    0.575 0.605]
 [0.    0.    0.    0.    0.    0.    0.    0.575 0.605]
 [0.    0.    0.    0.    0.    0.    0.    0.575 0.605]
 [0.    0.    0.    0.    0.    0.    0.    0.575 0.605]
 [0.    0.    0.    0.    0.    0.    0.    0.575 0.605]]
[[0.    0.    0.    0.    0.    0.    0.    0.    0.605]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.585]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.605]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.61 ]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.58 ]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.61 ]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.6  ]
 [0.    0.    0.    0.    0.  